In [ ]:
# currently testing in GCP AI Notebook
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image, ImageFile
import os
import numpy as np


In [ ]:
print (tf.__version__)




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator ##Data augmentation, and efficient labeling
TRAIN_DIR = "/home/jupyter/_train"
VALIDATION_DIR = "/home/jupyter/_validation"
TEST_DIR = "/home/jupyter/_test"
BATCH_SIZE = 64




In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 30,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                   target_size=(380,380))
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                             batch_size=BATCH_SIZE,
                                                             class_mode = 'categorical',
                                                             target_size=(380,380))



In [ ]:
module_selection = ("efficientnet", 380)
handle_base, pixels = module_selection
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/lite4/feature-vector/2"
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

In [ ]:
import tensorflow_hub as hub
model = keras.Sequential([
    keras.layers.InputLayer(input_shape = IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable = False),
    keras.layers.Dropout(rate = 0.25),
    keras.layers.Dense(64,
                      activation = "relu",
                      kernel_regularizer = keras.regularizers.l2(0.001)),
    keras.layers.Dense(8,
                      activation = "softmax")
])
model.build((None,) + IMAGE_SIZE + (3,))
model.summary()




In [ ]:
model.compile(optimizer = keras.optimizers.Adam(lr=0.0001),
             loss = keras.losses.categorical_crossentropy,
             metrics = ['accuracy'])




In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit(train_generator,
                   epochs = 15,
                   verbose = 1,
                   validation_data = validation_generator)


